In [26]:
import requests
from requests.compat import urljoin
import json
import shutil
import pandas as pd
import io
import re
import numpy as np

In [27]:
base="https://liangfgithub.github.io/MovieData/"
urlmovies=urljoin(base,"movies.dat?raw=true")
urlratings=urljoin(base,"ratings.dat?raw=true")
urlusers=urljoin(base,"users.dat?raw=true")
baseimages="https://liangfgithub.github.io/MovieImages/"

print (urlmovies)

https://liangfgithub.github.io/MovieData/movies.dat?raw=true


In [28]:
r=requests.get(urlmovies,stream = True)
data=r.text
moviesraw=pd.read_csv(io.StringIO(data),sep='::',header=None)
moviesraw.columns=["MovieID","Movie","Genres"]
moviesraw

,MovieID,Movie,Genres
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy
...,...,...,...
3878,3948,Meet the Parents (2000),Comedy
3879,3949,Requiem for a Dream (2000),Drama
3880,3950,Tigerland (2000),Drama
3881,3951,Two Family House (2000),Drama


In [29]:
movies=moviesraw
movies.Movie=movies.Movie.str.replace(r"\(.*\) ?\(","(")
movies[["Title","Year"]]=moviesraw.Movie.str.split(r" \(",expand=True,n=1)
movies.Year=movies.Year.str.replace(")","").astype(int)
movies.Title=movies.Title.str.replace(r"[^a-zA-Z ]","")
movies['Filename']="images/movies/"+movies.MovieID.astype(str)+".jpg"
movies[["Cat1","Cat2","Cat3","Cat4","Cat5","Cat6"]]=movies.Genres.str.split(r"|",expand=True,n=5)
movies

,MovieID,Movie,Genres,Title,Year,Filename,Cat1,Cat2,Cat3,Cat4,Cat5,Cat6
0,1,Toy Story (1995),Animation|Children's|Comedy,Toy Story,1995,images/movies/1.jpg,Animation,Children's,Comedy,None,None,None
1,2,Jumanji (1995),Adventure|Children's|Fantasy,Jumanji,1995,images/movies/2.jpg,Adventure,Children's,Fantasy,None,None,None
2,3,Grumpier Old Men (1995),Comedy|Romance,Grumpier Old Men,1995,images/movies/3.jpg,Comedy,Romance,None,None,None,None
3,4,Waiting to Exhale (1995),Comedy|Drama,Waiting to Exhale,1995,images/movies/4.jpg,Comedy,Drama,None,None,None,None
4,5,Father of the Bride Part II (1995),Comedy,Father of the Bride Part II,1995,images/movies/5.jpg,Comedy,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...
3878,3948,Meet the Parents (2000),Comedy,Meet the Parents,2000,images/movies/3948.jpg,Comedy,None,None,None,None,None
3879,3949,Requiem for a Dream (2000),Drama,Requiem for a Dream,2000,images/movies/3949.jpg,Drama,None,None,None,None,None
3880,3950,Tigerland (2000),Drama,Tigerland,2000,images/movies/3950.jpg,Drama,None,None,None,None,None
3881,3951,Two Family House (2000),Drama,Two Family House,2000,images/movies/3951.jpg,Drama,None,None,None,None,None


In [30]:
r=requests.get(urlratings,stream = True)
data=r.text
ratingsraw=pd.read_csv(io.StringIO(data),sep='::',header=None)
ratingsraw.columns=["UserID", "MovieID", "Rating", "Timestamp"]
ratingsraw

,UserID,MovieID,Rating,Timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291
...,...,...,...,...
1000204,6040,1091,1,956716541
1000205,6040,1094,5,956704887
1000206,6040,562,5,956704746
1000207,6040,1096,4,956715648


In [31]:
ratings=ratingsraw
ratingmean=ratings[['MovieID','Rating']].groupby('MovieID').mean('Rating')
ratingmean

,Rating
MovieID,
1,4.146846
2,3.201141
3,3.016736
4,2.729412
5,3.006757
...,...
3948,3.635731
3949,4.115132
3950,3.666667


In [32]:
cat_cols = [col for col in movies.columns if 'Cat' in col]
print(cat_cols)
categories= pd.DataFrame(columns=['MovieID','Category'])
for cat_col in cat_cols:
    catadd=movies.dropna(subset=[cat_col])[["MovieID",cat_col]]
    catadd.columns=["MovieID",'Category']
    categories=categories.append(catadd, ignore_index=True)
categories=categories.set_index('MovieID')
categories

['Cat1', 'Cat2', 'Cat3', 'Cat4', 'Cat5', 'Cat6']


,Category
MovieID,
1,Animation
2,Adventure
3,Comedy
4,Comedy
5,Comedy
...,...
2054,Sci-Fi
2080,Romance
2081,Romance


In [34]:
categoriesrating=categories.join(ratingmean)
categoriesrating

,Category,Rating
MovieID,,
1,Animation,4.146846
1,Children's,4.146846
1,Comedy,4.146846
2,Adventure,3.201141
2,Children's,3.201141
...,...,...
3949,Drama,4.115132
3950,Drama,3.666667
3951,Drama,3.900000


In [35]:
catmatrix=categoriesrating.pivot(columns='Category',values='Rating')
catmatrix

Category,Action,Adventure,Animation,Children's,Comedy,Crime,Documentary,Drama,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
MovieID,,,,,,,,,,,,,,,,,,
1,NaN,NaN,4.146846,4.146846,4.146846,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,3.201141,NaN,3.201141,NaN,NaN,NaN,NaN,3.201141,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,3.016736,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.016736,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,2.729412,NaN,NaN,2.729412,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,3.006757,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3948,NaN,NaN,NaN,NaN,3.635731,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3949,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.115132,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3950,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.666667,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [36]:
Category='Drama'
catmatrix[Category].sort_values(ascending=False)[:20]

MovieID
989     5.000000
3382    5.000000
3607    5.000000
3245    4.800000
53      4.750000
2503    4.666667
2019    4.560510
318     4.554558
858     4.524966
527     4.510417
3888    4.500000
2480    4.500000
439     4.500000
2309    4.500000
1795    4.500000
3517    4.500000
578     4.500000
557     4.500000
1178    4.473913
1207    4.425647
Name: Drama, dtype: float64

In [59]:
#categories=categories.reset_index()
ucat=categories.Category.unique()
ucat.sort()
ucategories=pd.DataFrame(data=ucat,columns=['Category'])
ucategories

,Category
0,Action
1,Adventure
2,Animation
3,Children's
4,Comedy
5,Crime
6,Documentary
7,Drama
8,Fantasy
9,Film-Noir


In [61]:
ucategories.to_feather('./recommenderapp/static/categories.feather')

In [266]:
catmatrix=catmatrix.reset_index()

In [267]:
catmatrix.to_feather('./recommenderapp/static/genresmatrix.feather')

In [12]:
movies=movies.reset_index()
movies=movies[['MovieID','Movie','Title','Year','Filename']]
movies

,MovieID,Movie,Title,Year,Filename
0,1,Toy Story (1995),Toy Story,1995,images/movies/1.jpg
1,2,Jumanji (1995),Jumanji,1995,images/movies/2.jpg
2,3,Grumpier Old Men (1995),Grumpier Old Men,1995,images/movies/3.jpg
3,4,Waiting to Exhale (1995),Waiting to Exhale,1995,images/movies/4.jpg
4,5,Father of the Bride Part II (1995),Father of the Bride Part II,1995,images/movies/5.jpg
...,...,...,...,...,...
3878,3948,Meet the Parents (2000),Meet the Parents,2000,images/movies/3948.jpg
3879,3949,Requiem for a Dream (2000),Requiem for a Dream,2000,images/movies/3949.jpg
3880,3950,Tigerland (2000),Tigerland,2000,images/movies/3950.jpg
3881,3951,Two Family House (2000),Two Family House,2000,images/movies/3951.jpg


In [13]:
movies.to_feather('./recommenderapp/static/movies.feather')